# Rapport : Dimensionnement d'un Buffer Réseau

<div align="center">

## Projet de Modélisation par Chaînes de Markov à Temps Discret

**Cours :** File d'attente, TSP FISA 2A  
**Auteurs :** Nassim EL HADDAD, Rebecca RINGUET  
**Date :** 2025

</div>

---

## Table des Matières

1. [Introduction et Contexte](#introduction)
2. [Modélisation du Système](#modelisation)
3. [Question 1 : Chaîne de Markov](#q1)
4. [Question 2 : Distribution Stationnaire](#q2)
5. [Question 3 : Nombre Moyen de Clients](#q3)
6. [Question 4 : Débit d'Entrée](#q4)
7. [Question 5 : Taux de Perte](#q5)
8. [Question 6 : Temps de Réponse Moyen](#q6)
9. [Question 7 : Taux d'Utilisation](#q7)
10. [Question 8 : Analyse Multi-Capacités](#q8)
11. [Question 9 : Analyse et Commentaires](#q9)
12. [Question 10 : Comparaison de Configurations](#q10)
13. [Conclusion](#conclusion)

---




In [ ]:
<a id="introduction"></a>
## 1. Introduction et Contexte

Ce projet traite du dimensionnement d'un buffer dans un nœud réseau en utilisant une modélisation par chaînes de Markov à temps discret. L'objectif est d'analyser les performances d'un système de file d'attente avec une capacité finie et de déterminer les métriques clés telles que le nombre moyen de clients, le débit d'entrée, le taux de perte, le temps de réponse et le taux d'utilisation.

### Paramètres du Système

- **Taille des paquets** : 64 octets
- **Débit du lien** : 512 Mbits/s
- **Unité de temps** : slot de 10⁻⁶ s (1 microseconde)
- **Service** : 1 paquet servi par slot
- **Capacité du buffer** : K (finie)

### Distribution d'Arrivées - Configuration Initiale

- **p₀ = 0.4** : Probabilité de 0 arrivée par slot
- **p₁ = 0.2** : Probabilité de 1 arrivée par slot
- **p₂ = 0.4** : Probabilité de 2 arrivées par slot

---


<a id="modelisation"></a>
## 2. Modélisation du Système

### Principe de la Chaîne de Markov

Une **chaîne de Markov à temps discret** est utilisée pour modéliser l'évolution du nombre de paquets dans le buffer au cours du temps.

#### États du Système
- **État i** : Nombre de paquets dans le buffer (i ∈ {0, 1, 2, ..., K})
- **K+1 états** possibles au total

#### Transitions d'État

Pour chaque état **i**, le système évolue selon :

1. **Service** : 1 paquet est servi (si i > 0)
   - État après service : $\max(0, i-1)$

2. **Arrivées** : 0, 1 ou 2 paquets arrivent selon les probabilités
   - Nouvel état : $\min(K, \text{état\_après\_service} + \text{arrivées})$
   - Les paquets en excès sont **rejetés** si le buffer est plein

#### Matrice de Transition P

La matrice **P** de taille $(K+1) \times (K+1)$ représente les probabilités de transition :

$$P_{ij} = \mathbb{P}(X_{n+1} = j \mid X_n = i)$$

où $X_n$ est l'état du système au slot $n$.

### Métriques de Performance

Le projet calcule les métriques suivantes :

#### Distribution Stationnaire π

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue par la formule analytique :

$$\pi_i = \pi_0 \cdot \left(\frac{p_2}{p_0}\right)^i$$

avec la contrainte de normalisation :

$$\sum_{i=0}^{K} \pi_i = 1$$

#### Nombre Moyen de Clients L

$$L = \sum_{i=1}^{K} i \cdot \pi_i$$

#### Débit d'Entrée Xe

$$X_e = 2 \cdot p_2 \cdot \sum_{i=0}^{K-1} \pi_i + 1 \cdot \left(p_1 \cdot \sum_{i=0}^{K} \pi_i + p_2 \cdot \pi_K\right)$$

#### Taux de Perte Loss

$$\text{Loss} = 1 \cdot p_2 \cdot \pi_K$$

#### Temps de Réponse Moyen R (Formule de Little)

$$R = \frac{L}{X_e}$$

#### Taux d'Utilisation U

$$U = 1 - \pi_0$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien soit occupé.

---


In [ ]:
# Configuration et imports
import numpy as np
from scipy.linalg import solve
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import networkx as nx
import pandas as pd
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

# Configuration du style matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Couleurs personnalisées pour les graphiques
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'success': '#06A77D',
    'warning': '#F18F01',
    'danger': '#C73E1D',
    'info': '#6C5CE7'
}

print("Bibliothèques importées avec succès !")
print("Configuration des graphiques appliquée")


In [ ]:
# Chargement des fonctions du projet
exec(open('buffer_dimensionnement.py').read())

print("Fonctions du projet chargées avec succès !")


<a id="q1"></a>
## 3. Question 1 : Chaîne de Markov

### Description de la Chaîne

Pour **K = 5**, la chaîne de Markov possède **6 états** (0 à 5) représentant le nombre de paquets dans le buffer.

Le graphe suivant montre la structure de la chaîne de Markov avec les transitions possibles entre les états :

![Chaîne de Markov pour K=5](graphes/chaineDeMarkov.jpeg)

**Commentaire :** Le graphe montre les 6 états (0 à 5) et les transitions possibles. Les arcs sont étiquetés avec les probabilités de transition. On observe que :
- L'état 0 ne peut que recevoir des paquets (pas de service)
- Les états intermédiaires peuvent recevoir ou perdre des paquets
- L'état 5 (buffer plein) peut perdre des paquets lors d'arrivées

---


In [ ]:
# Cette cellule est maintenant vide car l'image est intégrée directement dans la cellule markdown précédente


<a id="q2"></a>
## 4. Question 2 : Distribution de Probabilité Stationnaire π(K)

### Calcul de la Distribution Stationnaire

La distribution stationnaire $\pi = [\pi_0, \pi_1, \ldots, \pi_K]$ est obtenue par la formule analytique :

$$\pi_i = \pi_0 \cdot \left(\frac{p_2}{p_0}\right)^i$$

avec la contrainte de normalisation :

$$\sum_{i=0}^{K} \pi_i = 1$$

Cette distribution représente les **probabilités d'équilibre** de chaque état du système à long terme, c'est-à-dire :

$$\pi_i = \lim_{n \to \infty} \mathbb{P}(X_n = i)$$

### Visualisation pour K ∈ {2, 5, 10, 15, 80}

Le graphique suivant montre la distribution stationnaire pour différentes valeurs de K avec la configuration initiale (p₀=0.4, p₁=0.2, p₂=0.4) :

![Distribution stationnaire - Configuration originale](graphes/Q2_distribution_stationnaire_Original.png)

**Observations :**
- Pour K petit (K=2, 5) : La probabilité est concentrée sur les états faibles
- Pour K moyen (K=10, 15) : La distribution s'étale davantage
- Pour K grand (K=80) : La distribution se stabilise avec une queue décroissante
- L'état 0 (buffer vide) a toujours une probabilité significative

---

In [ ]:
# Configuration et imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Chargement des fonctions du projet
exec(open('buffer_dimensionnement.py').read())

# Paramètres initiaux
p0, p1, p2 = 0.4, 0.2, 0.4
valeurs_K = [2, 5, 10, 15, 80]

print("Bibliothèques et fonctions chargées avec succès !")


<a id="q3"></a>
## 5. Question 3 : Nombre Moyen de Clients L

### Définition

Le nombre moyen de clients dans le système est défini par :

$$L = \sum_{i=1}^{K} i \cdot \pi_i$$

Cette métrique représente le nombre moyen de paquets présents dans le système (buffer) en régime stationnaire.

---


In [ ]:
# Calcul des métriques pour toutes les valeurs de K
resultats_complets = []

for K in valeurs_K:
    resultat = analyser_buffer(K, p0, p1, p2, verbose=False)
    if resultat:
        resultats_complets.append(resultat)

# Extraction des valeurs
K_values = [r['K'] for r in resultats_complets]
L_values = [r['L'] for r in resultats_complets]
Xe_values = [r['Xe'] for r in resultats_complets]
Loss_values = [r['Loss'] for r in resultats_complets]
R_values = [r['R'] for r in resultats_complets]
U_values = [r['U']*100 for r in resultats_complets]

print("Calculs terminés pour toutes les valeurs de K.")


<a id="q4"></a>
## 6. Question 4 : Débit d'Entrée Xe

### Définition

Le débit d'entrée Xe représente le nombre moyen de paquets acceptés par slot. Il est calculé par :

$$X_e = 2 \cdot p_2 \cdot \sum_{i=0}^{K-1} \pi_i + 1 \cdot \left(p_1 \cdot \sum_{i=0}^{K} \pi_i + p_2 \cdot \pi_K\right)$$

Cette métrique augmente avec K car moins de paquets sont rejetés lorsque la capacité augmente.

---


<a id="q5"></a>
## 7. Question 5 : Taux de Perte Loss

### Définition

Le taux de perte représente le nombre moyen de paquets perdus par slot. Il est calculé par :

$$\text{Loss} = 1 \cdot p_2 \cdot \pi_K$$

Les paquets sont perdus lorsque le buffer est plein (état K) et qu'il y a une arrivée de 2 paquets. Cette métrique diminue exponentiellement avec K.

---


In [ ]:
<a id="q6"></a>
## 8. Question 6 : Temps de Réponse Moyen R

### Définition

Le temps de réponse moyen est calculé à l'aide de la formule de Little :

$$R = \frac{L}{X_e}$$

Cette formule fondamentale de la théorie des files d'attente relie le nombre moyen de clients dans le système, le débit d'entrée et le temps de réponse moyen. Le temps de réponse augmente avec K car plus de paquets sont stockés en moyenne.

---


<a id="q7"></a>
## 9. Question 7 : Taux d'Utilisation U

### Définition

Le taux d'utilisation du lien est calculé par :

$$U = 1 - \pi_0 = \sum_{i=1}^{K} \pi_i$$

Le taux d'utilisation représente la probabilité que le buffer soit non vide, c'est-à-dire que le lien de transmission soit occupé. Cette métrique augmente avec K.

---

### Visualisation des Métriques Q3-Q7

Le graphique suivant montre l'évolution de toutes les métriques (L, Xe, Loss, R, U) en fonction de K pour la configuration initiale :

![Métriques en fonction de K - Configuration originale](graphes/Q3-Q7_metriques_K_Original.png)

**Observations :**
- **L (nombre moyen de clients)** : Augmente avec K, plus de capacité = plus de paquets stockés
- **Xe (débit d'entrée)** : Augmente avec K, converge vers le débit théorique d'arrivée (1.0 paquets/slot)
- **Loss (taux de perte)** : Diminue exponentiellement avec K (échelle logarithmique)
- **R (temps de réponse)** : Augmente avec K, trade-off entre pertes et délai
- **U (taux d'utilisation)** : Augmente avec K, reflète la probabilité que le buffer soit non vide

---


<a id="q8"></a>
## 10. Question 8 : Analyse Multi-Capacités

### Analyse pour K ∈ {2, 5, 10, 15, 80}

Cette section présente une analyse comparative détaillée pour différentes capacités de buffer.

### Tableau Récapitulatif des Métriques

| K | L (paquets) | Xe (paquets/slot) | Loss (paquets/slot) | R (slots) | U (%) |
|---|-------------|-------------------|---------------------|-----------|-------|
| 2 | 1.000000 | 0.866667 | 0.133333 | 1.153846 | 66.67 |
| 5 | 2.500000 | 0.933333 | 0.066667 | 2.678571 | 83.33 |
| 10 | 5.000000 | 0.963636 | 0.036364 | 5.188712 | 90.91 |
| 15 | 7.500000 | 0.975000 | 0.025000 | 7.692308 | 93.75 |
| 80 | 40.000000 | 0.995062 | 0.004938 | 40.198501 | 98.77 |

### Observations Principales

- **Pour K petit (K=2, 5)** : Taux de perte élevé, débit d'entrée limité
- **Pour K moyen (K=10, 15)** : Bon compromis entre pertes et délai
- **Pour K grand (K=80)** : Taux de perte négligeable, mais temps de réponse plus élevé

---


<a id="q10"></a>
## 12. Question 10 : Comparaison de Configurations

### Configuration Modifiée

Selon le sujet, nous modifions les probabilités d'arrivée : **p₀=0.5, p₁=0.2, p₂=0.3**

| Configuration | p₀ | p₁ | p₂ |
|---------------|----|----|----|
| **Originale** | 0.4 | 0.2 | 0.4 |
| **Modifiée** | 0.5 | 0.2 | 0.3 |

### Changements Clés

- **p₀ augmente** : Plus de slots sans arrivée (0.4 → 0.5)
- **p₂ diminue** : Moins d'arrivées de 2 paquets (0.4 → 0.3)

### Impact Attendu

- **Débit d'arrivée théorique réduit** : 1.0 → 0.8 paquets/slot (0.5×0 + 0.2×1 + 0.3×2 = 0.8)
- **Plus de périodes creuses** : p₀ plus élevé permet au buffer de se vider plus souvent
- **Moins de pics** : p₂ réduit diminue les arrivées simultanées de 2 paquets

### Comparaison Visuelle

Le graphique suivant compare les métriques entre les deux configurations :

![Comparaison des configurations](graphes/Q10_comparaison_configurations.png)

**Observations principales :**
- **Configuration modifiée** : Débit théorique réduit (0.8 vs 1.0 paquets/slot)
- **Pour K petit** : Moins de congestion grâce à moins d'arrivées simultanées
- **Pour K grand** : Meilleure récupération grâce à p₀=0.5 (plus de slots vides)
- **Temps de réponse** : Généralement plus faible avec la configuration modifiée

---


In [ ]:
# Calcul avec la configuration modifiée (pour générer les graphiques si nécessaire)
p0_new, p1_new, p2_new = 0.5, 0.2, 0.3

resultats_nouvelle = []
for K in valeurs_K:
    resultat = analyser_buffer(K, p0_new, p1_new, p2_new, verbose=False)
    if resultat:
        resultats_nouvelle.append(resultat)

print("Calculs pour la configuration modifiée terminés.")


<a id="conclusion"></a>
## 13. Conclusion et Recommandations

### Synthèse des Résultats

#### Impact de la Capacité K

1. **Petites capacités (K = 2, 5)**
   - Taux de perte élevé
   - Débit d'entrée limité
   - Temps de réponse faible (peu de paquets en attente)
   - **Recommandation** : Inadapté pour des applications sensibles aux pertes

2. **Capacités moyennes (K = 10, 15)**
   - Bon compromis entre pertes et délai
   - Débit d'entrée proche du théorique
   - Temps de réponse acceptable
   - **Recommandation** : Optimal pour la plupart des applications

3. **Grandes capacités (K = 80)**
   - Taux de perte négligeable
   - Débit d'entrée maximal
   - Temps de réponse plus élevé
   - **Recommandation** : Adapté pour applications tolérantes au délai mais sensibles aux pertes

#### Comparaison des Configurations

- **Configuration originale** : Plus stable, moins de pics, débit théorique de 1.0 paquets/slot
- **Configuration modifiée** : Moins de pics, débit théorique réduit à 0.8 paquets/slot, meilleure récupération
- **Recommandation** : Choisir selon les contraintes de l'application

### Recommandations Finales

1. **Pour minimiser les pertes** : Utiliser K ≥ 15
2. **Pour minimiser le délai** : Utiliser K ≤ 10
3. **Pour un compromis optimal** : K = 10-15
4. **Pour la configuration** : Préférer l'originale pour plus de stabilité et un débit plus élevé

---

<div align="center">

### Projet réalisé dans le cadre du cours de File d'attente, TSP FISA 2A

**Auteurs :**  
Nassim EL HADDAD  
Rebecca RINGUET

**Merci de votre attention !**

</div>
